In [1]:
import awkward as ak
import numpy as np

import hepfile

import uproot

import time

print(ak.__version__)
print(uproot.__version__)

2.3.1
5.0.10


In [2]:
# Down load a file for us to play with
!curl http://opendata.cern.ch/record/12361/files/SMHiggsToZZTo4L.root --output SMHiggsToZZTo4L.root

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.4M  100 40.4M    0     0  13.9M      0  0:00:02  0:00:02 --:--:-- 13.9M


In [3]:
# This is all for demonstration purposes, to show people how this type of
# writing could be done. 
# But of course people could just create their own awkward arrays.

f = uproot.open('SMHiggsToZZTo4L.root')

events = f['Events']

events

<TTree 'Events' (32 branches) at 0x7f7a9c04d460>

In [4]:
# What's in this file? 
events.keys()

['run',
 'luminosityBlock',
 'event',
 'PV_npvs',
 'PV_x',
 'PV_y',
 'PV_z',
 'nMuon',
 'Muon_pt',
 'Muon_eta',
 'Muon_phi',
 'Muon_mass',
 'Muon_charge',
 'Muon_pfRelIso03_all',
 'Muon_pfRelIso04_all',
 'Muon_dxy',
 'Muon_dxyErr',
 'Muon_dz',
 'Muon_dzErr',
 'nElectron',
 'Electron_pt',
 'Electron_eta',
 'Electron_phi',
 'Electron_mass',
 'Electron_charge',
 'Electron_pfRelIso03_all',
 'Electron_dxy',
 'Electron_dxyErr',
 'Electron_dz',
 'Electron_dzErr',
 'MET_pt',
 'MET_phi']

In [5]:
# Let's check out these types and their sizes

print(type(events['Muon_pt'].array()))
print()

# Number of events
print(ak.num(events['Muon_pt'].array(),axis=0))

# Number of muons in each event
print(ak.num(events['Muon_pt'].array(),axis=1))

# Number of muons in total
print(ak.sum(ak.num(events['Muon_pt'].array(),axis=1)))

<class 'awkward.highlevel.Array'>

299973
[3, 0, 0, 7, 0, 2, 1, 0, 0, 0, 1, 2, 0, ..., 0, 0, 0, 2, 2, 2, 0, 2, 2, 4, 0, 0]
441394


In [6]:
# While not all the entries in the ROOT file naturally lend themselves to 
# group/dataset breakdowns, some do. Let's find those "automatically", 
# just to make it easier to write them to the hepfile.

# Find groups
def make_groups_and_datasets(fields):
    
    groups = {}
    
    for field in fields:
        if field.find('_')>=0:
            
            # Do this in case there is more than one underscore
            idx = field.find('_')
            
            #print(field)
            grp = field[0:idx]
            dset = field[idx+1:]
            
            if grp not in groups.keys():
                groups[grp] = [[field,dset]]
            else:
                groups[grp].append([field,dset])
    
    return groups


############################################################

groupings = make_groups_and_datasets(events.keys())

# Groupings gives us a nice mapping of the names from the ROOT file
# to how we're going to store them in our hepfile as 
# group/datasets

print(groupings)
print()
print(groupings['Muon'])

{'PV': [['PV_npvs', 'npvs'], ['PV_x', 'x'], ['PV_y', 'y'], ['PV_z', 'z']], 'Muon': [['Muon_pt', 'pt'], ['Muon_eta', 'eta'], ['Muon_phi', 'phi'], ['Muon_mass', 'mass'], ['Muon_charge', 'charge'], ['Muon_pfRelIso03_all', 'pfRelIso03_all'], ['Muon_pfRelIso04_all', 'pfRelIso04_all'], ['Muon_dxy', 'dxy'], ['Muon_dxyErr', 'dxyErr'], ['Muon_dz', 'dz'], ['Muon_dzErr', 'dzErr']], 'Electron': [['Electron_pt', 'pt'], ['Electron_eta', 'eta'], ['Electron_phi', 'phi'], ['Electron_mass', 'mass'], ['Electron_charge', 'charge'], ['Electron_pfRelIso03_all', 'pfRelIso03_all'], ['Electron_dxy', 'dxy'], ['Electron_dxyErr', 'dxyErr'], ['Electron_dz', 'dz'], ['Electron_dzErr', 'dzErr']], 'MET': [['MET_pt', 'pt'], ['MET_phi', 'phi']]}

[['Muon_pt', 'pt'], ['Muon_eta', 'eta'], ['Muon_phi', 'phi'], ['Muon_mass', 'mass'], ['Muon_charge', 'charge'], ['Muon_pfRelIso03_all', 'pfRelIso03_all'], ['Muon_pfRelIso04_all', 'pfRelIso04_all'], ['Muon_dxy', 'dxy'], ['Muon_dxyErr', 'dxyErr'], ['Muon_dz', 'dz'], ['Muon_dzErr'

In [7]:
# There are some others. THese will be SINGLETONS that we pass in separately.
'run',
'luminosityBlock',
'event',

print(events['run'].array())
print(events['luminosityBlock'].array())
print(events['event'].array())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[156, 156, 156, 156, 156, 156, 156, 156, ..., 996, 996, 996, 996, 996, 996, 996]
[46501, 46502, 46503, 46504, 46505, ..., 298796, 298797, 298798, 298799, 298800]


In [8]:
def _get_awkward_type(ak_array: ak.Record) -> type:
    try:
        if isinstance(ak_array[0], (ak.Record, ak.Array)):
            arr = ak_array
            type_str = ak_array.type.content            
            
            if isinstance(type_str, ak.types.NumpyType):
                dtype = type_str.primitive
            else:
                dtype = str(type_str).rsplit("*", maxsplit=1)[-1].strip()
            
        else:
            arr = np.array(ak_array)
            dtype = arr.dtype

        if dtype == "string":
            dtype = np.dtype("<U1")
    
        np_dtype = np.dtype(dtype)
        if np_dtype.char == "U":
            np_dtype = str

    except Exception as exc:
        raise IOError("Cannot convert input value to a numpy data type!") from exc

    return np_dtype

# Try it out
print(_get_awkward_type(events['Muon_pt'].array()))
print(_get_awkward_type(events['MET_pt'].array()))
print(_get_awkward_type(events['run'].array()))

float32
float32
int32


In [66]:
# This is the function to pack a single awkward array

def pack_single_awkward_array(d, arr, dset_name, group_name=None, counter=None):
    '''
    Packs a 1D awkward array as a dataset/singleton depending on if group_name is given
    
    Args:
        d [dict]: data dictionary created by hepfile.initialize()
        arr [ak.Array]: 1D awkward array to pack as either a dataset or a group. If group_name is None
                        the arr is packed as a singleton
        dset_name [str]: Full path to the dataset.
        group_name [str]: name of the group to pack the arr under, default is None
        counter [str]: name of the counter in the hepfile for this dataset
    '''   
    counter_in_hepfile = False
    if group_name is not None:

        if counter is None:
            counter = f"{group_name}/n{group_name}"
        
        # add the counter to the groups dictionary if it is not already in it
        if group_name not in d['_GROUPS_']:
            d['_GROUPS_'][group_name] = [counter.split('/')[1]]

            # We will use this name for the counter later
            d['_MAP_DATASETS_TO_DATA_TYPES_'][counter] = int

            d['_MAP_DATASETS_TO_COUNTERS_'][group_name] = counter
            d['_LIST_OF_COUNTERS_'].append(counter)

    else:
        counter = '_SINGLETONS_GROUP_/COUNTER'

    # Tells us if this is jagged or not
    if arr.ndim == 1:
        
        # Get the datatpe before we flatten it
        dtype = _get_awkward_type(arr)
        
        num = np.ones(len(arr),dtype=int)
        
        x = ak.to_numpy(arr)

    else:

        # Get the datatpe before we flatten it
        dtype = _get_awkward_type(arr)

        # This saves the counter as int64, taking up a bit more space
        # Probably minimal though. 
        #num = ak.num(x)
        # This saves the counter as int32
        num = ak.to_numpy(ak.num(arr)).astype(np.int32)

        x = ak.flatten(arr).to_numpy()

    d[dset_name] = x

    # Not a SINGLETON, the user has passed in a groupname
    if group_name is not None:
        d['_MAP_DATASETS_TO_DATA_TYPES_'][dset_name] = dtype
        d['_MAP_DATASETS_TO_COUNTERS_'][dset_name] = counter
        d['_GROUPS_'][group_name].append(dset_name.split('/')[1])
        if counter not in d:
            d[counter] = num
        
    # If it is a SINGLETON
    else:
        d['_MAP_DATASETS_TO_DATA_TYPES_'][dset_name] = dtype
        d['_MAP_DATASETS_TO_COUNTERS_'][dset_name] = '_SINGLETONS_GROUP_/COUNTER'
        d['_GROUPS_']['_SINGLETONS_GROUP_'].append(dset_name)
        if len(d[counter]) == 0:
            d[counter] = num

In [99]:
# I changed the name of this function to pack multiple awkward arrays.
# I agree with you that we should probably also have a function that packs
# single awkward arrays that gets called by this function.

# d = data dictionary, maybe we have a version of this that automatically
# creates the dictionary and returns it?

# arr = dictionary of awkward arrays and keys to use as names for the dataset

# group_name = group the dataset will belong to

# Right now, I am creating a counter "n{group_name}", is that what we want?
# Or should we allow the user to pass in the counter name, like when
# create_group is called?
#
# Right now, it creates the group each time this function is called
# We should have it check to see if the group is already there and if so, 
# don't add it again.

import warnings

def pack_multiple_awkward_arrays(d, arr, group_name=None, group_counter_name=None):
    '''
    Pack an awkward array of arrays into group_name or the singletons group
    
    Args:
        d [dict]: hepfile data dictionary that is returned from hepfile.initialize()
        arr [ak.Array]: Awkward array of the group in a set of data
        group_name [str]: Name of the group to pack arr into, if None (default) it is 
                          packed into the signletons group
    '''
    
    # If the user passed in a group, then the datasets will 
    # be under that group
    
    # check that arr is an awkward array
    if not isinstance(arr, (ak.Array, ak.Record)):
        try:
            arr = ak.Array(arr)
        except Exception as exc:
            raise IOError() from exc
    
    if len(arr.fields) == 0:
        raise IOError('The input awkward array must have at least one field value! '+
                      'If this is a singleton just provide the name of the singleton as the field')
    # Loop over the dictionary that is passed in        
    for field in arr.fields:
        
        # build a name for the hepfile entry
        if group_name is None:
            # these are singletons
            dataset_name = field
        else:
            # these are regular groups with datasets
            dataset_name = f"{group_name}/{field}"
            
        pack_single_awkward_array(d, arr[field], dataset_name, group_name=group_name, counter=group_counter_name)

    # We don't need to return the dictionary because in python
    # dictionaries are mutable. The dictionary in the function points to 
    # the same object outside of the function.
###############################################################################
    

In [100]:
# Here's how we call it. Though maybe we have an option to have
# the data dictionary created inside the function? 

# Initialize the data dictionary
data = hepfile.initialize()

# Pack these groups of awkward arrays

# This is what it would look like "by hand"
# A dictionary with the name of the dataset as it is to appear inside the hepfile
# and then the actual awkward array (not just the Branch object returned by uproot)
#ak_arrays = {'pt': events['Muon_pt'].array(), 'eta': events['Muon_eta'].array(), 'phi':events['Muon_phi'].array(), }

# Here I'm packing all the data that are groups/datasets
for groups_to_write in ['Muon', 'Electron', 'MET', 'PV']:
    ak_arrays = {}
    for grouping in groupings[groups_to_write]:
        ak_arrays[grouping[1]] = events[grouping[0]].array()
    
    pack_multiple_awkward_arrays(data, ak_arrays, group_name=groups_to_write)

# Now the SINGLETONS
ak_arrays = {"run":events['run'].array(), \
             "luminosityBlock":events['luminosityBlock'].array(), \
             "event":events['event'].array()}

# Note that there is no group name passed in. 
pack_multiple_awkward_arrays(data, ak_arrays)

In [101]:
len(data['_SINGLETONS_GROUP_/COUNTER'])

299973

In [102]:
len(data['Electron/nElectron'])

299973

In [103]:
# Uncomment if you want to see what the data dictionary looks like
#data

print(data['_GROUPS_']['_SINGLETONS_GROUP_'])
print()

print(data['_MAP_DATASETS_TO_COUNTERS_'])
print()

data.keys()

['COUNTER', 'run', 'luminosityBlock', 'event']

{'_SINGLETONS_GROUP_': '_SINGLETONS_GROUP_/COUNTER', 'Muon': 'Muon/nMuon', 'Muon/pt': 'Muon/nMuon', 'Muon/eta': 'Muon/nMuon', 'Muon/phi': 'Muon/nMuon', 'Muon/mass': 'Muon/nMuon', 'Muon/charge': 'Muon/nMuon', 'Muon/pfRelIso03_all': 'Muon/nMuon', 'Muon/pfRelIso04_all': 'Muon/nMuon', 'Muon/dxy': 'Muon/nMuon', 'Muon/dxyErr': 'Muon/nMuon', 'Muon/dz': 'Muon/nMuon', 'Muon/dzErr': 'Muon/nMuon', 'Electron': 'Electron/nElectron', 'Electron/pt': 'Electron/nElectron', 'Electron/eta': 'Electron/nElectron', 'Electron/phi': 'Electron/nElectron', 'Electron/mass': 'Electron/nElectron', 'Electron/charge': 'Electron/nElectron', 'Electron/pfRelIso03_all': 'Electron/nElectron', 'Electron/dxy': 'Electron/nElectron', 'Electron/dxyErr': 'Electron/nElectron', 'Electron/dz': 'Electron/nElectron', 'Electron/dzErr': 'Electron/nElectron', 'MET': 'MET/nMET', 'MET/pt': 'MET/nMET', 'MET/phi': 'MET/nMET', 'PV': 'PV/nPV', 'PV/npvs': 'PV/nPV', 'PV/x': 'PV/nPV', 'PV/y': 'PV

dict_keys(['_GROUPS_', '_MAP_DATASETS_TO_COUNTERS_', '_LIST_OF_COUNTERS_', '_SINGLETONS_GROUP_/COUNTER', '_MAP_DATASETS_TO_DATA_TYPES_', '_META_', 'Muon/pt', 'Muon/nMuon', 'Muon/eta', 'Muon/phi', 'Muon/mass', 'Muon/charge', 'Muon/pfRelIso03_all', 'Muon/pfRelIso04_all', 'Muon/dxy', 'Muon/dxyErr', 'Muon/dz', 'Muon/dzErr', 'Electron/pt', 'Electron/nElectron', 'Electron/eta', 'Electron/phi', 'Electron/mass', 'Electron/charge', 'Electron/pfRelIso03_all', 'Electron/dxy', 'Electron/dxyErr', 'Electron/dz', 'Electron/dzErr', 'MET/pt', 'MET/nMET', 'MET/phi', 'PV/npvs', 'PV/nPV', 'PV/x', 'PV/y', 'PV/z', 'run', 'luminosityBlock', 'event'])

In [104]:
# Write it!

# Try it with no compression
start = time.time()
hepfile.write_to_file('awkward_write_test.h5', data, verbose=False)
dt_no_compression = time.time() - start

# Try it with compression!
start = time.time()
hepfile.write_to_file('awkward_write_test_COMP_gzip_OPT_9.h5', data, verbose=False, comp_type="gzip", comp_opts=9)
dt_with_compression = time.time() - start

print()
print()
print(f"Time to write uncompressed: {dt_no_compression}")
print(f"Time to write   compressed: {dt_with_compression}")

print()
print()

# Check out the sizes of the files!

!ls -ltr SMHiggsToZZTo4L.root
!ls -ltr awkward_write_test.h5
!ls -ltr awkward_write_test_COMP_gzip_OPT_9.h5



Time to write uncompressed: 0.05812954902648926
Time to write   compressed: 3.9766011238098145


-rw-r--r-- 1 nfranz nfranz 42400229 Jul 17 13:35 SMHiggsToZZTo4L.root
-rw-r--r-- 1 nfranz nfranz 60385968 Jul 17 14:12 awkward_write_test.h5
-rw-r--r-- 1 nfranz nfranz 33736637 Jul 17 14:12 awkward_write_test_COMP_gzip_OPT_9.h5


In [105]:
data,bucket = hepfile.load('awkward_write_test_COMP_gzip_OPT_9.h5', verbose=False)

#data,bucket = hepfile.load('awkward_write_test.h5')

In [106]:
data['Electron/nElectron']

array([0, 4, 2, ..., 0, 2, 2])

In [107]:
print(type(data['Muon/pt']))
print(type(data['Muon/pt'][0]))

<class 'numpy.ndarray'>
<class 'numpy.float32'>


In [108]:
type(data['Muon/pt'].astype(np.float32)[0])

numpy.float32

In [109]:
data['run']

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [110]:
data['_MAP_DATASETS_TO_DATA_TYPES_']

{'Electron/charge': dtype('int32'),
 'Electron/dxy': dtype('float32'),
 'Electron/dxyErr': dtype('float32'),
 'Electron/dz': dtype('float32'),
 'Electron/dzErr': dtype('float32'),
 'Electron/eta': dtype('float32'),
 'Electron/mass': dtype('float32'),
 'Electron/nElectron': dtype('int64'),
 'Electron/pfRelIso03_all': dtype('float32'),
 'Electron/phi': dtype('float32'),
 'Electron/pt': dtype('float32'),
 'MET/nMET': dtype('int64'),
 'MET/phi': dtype('float32'),
 'MET/pt': dtype('float32'),
 'Muon/charge': dtype('int32'),
 'Muon/dxy': dtype('float32'),
 'Muon/dxyErr': dtype('float32'),
 'Muon/dz': dtype('float32'),
 'Muon/dzErr': dtype('float32'),
 'Muon/eta': dtype('float32'),
 'Muon/mass': dtype('float32'),
 'Muon/nMuon': dtype('int64'),
 'Muon/pfRelIso03_all': dtype('float32'),
 'Muon/pfRelIso04_all': dtype('float32'),
 'Muon/phi': dtype('float32'),
 'Muon/pt': dtype('float32'),
 'PV/nPV': dtype('int64'),
 'PV/npvs': dtype('int32'),
 'PV/x': dtype('float32'),
 'PV/y': dtype('float32'),

In [111]:
data['_GROUPS_']

{'_SINGLETONS_GROUP_': ['run', 'luminosityBlock', 'event'],
 'Electron': ['charge',
  'dxy',
  'dxyErr',
  'dz',
  'dzErr',
  'eta',
  'mass',
  'nElectron',
  'pfRelIso03_all',
  'phi',
  'pt'],
 'MET': ['nMET', 'phi', 'pt'],
 'Muon': ['charge',
  'dxy',
  'dxyErr',
  'dz',
  'dzErr',
  'eta',
  'mass',
  'nMuon',
  'pfRelIso03_all',
  'pfRelIso04_all',
  'phi',
  'pt'],
 'PV': ['nPV', 'npvs', 'x', 'y', 'z']}

In [112]:
data.keys()

dict_keys(['_MAP_DATASETS_TO_COUNTERS_', '_MAP_DATASETS_TO_INDEX_', '_LIST_OF_COUNTERS_', '_LIST_OF_DATASETS_', '_META_', '_NUMBER_OF_BUCKETS_', '_SINGLETONS_GROUP_', '_SINGLETONS_GROUP_/COUNTER', 'Electron/nElectron', 'Electron/nElectron_INDEX', 'MET/nMET', 'MET/nMET_INDEX', 'Muon/nMuon', 'Muon/nMuon_INDEX', 'PV/nPV', 'PV/nPV_INDEX', '_SINGLETONS_GROUP_/COUNTER_INDEX', 'Electron/charge', 'Electron/dxy', 'Electron/dxyErr', 'Electron/dz', 'Electron/dzErr', 'Electron/eta', 'Electron/mass', 'Electron/pfRelIso03_all', 'Electron/phi', 'Electron/pt', 'MET/phi', 'MET/pt', 'Muon/charge', 'Muon/dxy', 'Muon/dxyErr', 'Muon/dz', 'Muon/dzErr', 'Muon/eta', 'Muon/mass', 'Muon/pfRelIso03_all', 'Muon/pfRelIso04_all', 'Muon/phi', 'Muon/pt', 'PV/npvs', 'PV/x', 'PV/y', 'PV/z', 'event', 'luminosityBlock', 'run', '_GROUPS_', '_MAP_DATASETS_TO_DATA_TYPES_', '_PROTECTED_NAMES_'])

In [81]:
x = ak.to_numpy(ak.flatten(events['Muon_pt'].array()))

type(x)
type(x[0])

numpy.float32

In [82]:
x = events['Muon_pt'].array()

xnum = ak.num(x)

print(type(xnum))
print(type(xnum[0]))

print(xnum)

print(xnum[0])

print(type(ak.to_numpy(xnum)[0]))
print(type(ak.to_numpy(xnum).astype(np.int32)[0]))

<class 'awkward.highlevel.Array'>
<class 'numpy.int64'>
[3, 0, 0, 7, 0, 2, 1, 0, 0, 0, 1, 2, 0, ..., 0, 0, 0, 2, 2, 2, 0, 2, 2, 4, 0, 0]
3
<class 'numpy.int64'>
<class 'numpy.int32'>


In [83]:
# test loading the hepfile into the awkward array
awk, bucket = hepfile.load('awkward_write_test_COMP_gzip_OPT_9.h5', return_type='awkward')
awk

<Array [{run: 1, ...}, ..., {run: 1, ...}] type='299973 * {run: int32, lumi...'>

In [115]:
def awkward_to_hepfile(ak_array, outfile=None, write_hepfile=True, **kwargs):
    '''
    Write an awkward array with depth <= 2 to a hepfile
    
    Args:
        ak_array [ak.Array]: awkward array with fields of groups/singletons. Under the group fields
                             are the dataset fields.
        outfile [str]: path to where the hepfile should be written. Default is None and can only be
                       None if write_hepfile=False.
        write_hepfile [bool]: if True, write the hepfile and return the data dictionary. If False, 
                              just return the data dictionary without returning. Default is True.
        **kwargs: passed to `hepfile.write_to_file`
                              
    Returns:
        Data dictionary in the hepfile
    '''
    
    # _is_valid_awkward(ak_array) # uncomment when in actual software and not in testing
    
    if write_hepfile is True and outfile is None:
        raise IOError('Please provide an outfile path if write_hepfile=True!')
        
    if write_hepfile is False and outfile is not None:
        warnings.warn(
            "You set write_hepfile to False but provided an output file path. \
            This output file path will not be used!"
        )
    
    data = hepfile.initialize()
    for group in ak_array.fields:
        
        if len(ak_array[group].fields) == 0:
            # this is a singleton
            pack_multiple_awkward_arrays(data, {group: ak_array[group]})
        else:
            # these are datasets under group
            pack_multiple_awkward_arrays(data, ak_array[group], group_name=group)
            
    if write_hepfile:
        hepfile.write_to_file(outfile, data)
    
    return data
    

In [116]:
# test awkward_to_hepfile

d2 = awkward_to_hepfile(awk, write_hepfile=False)
d2

{'_GROUPS_': {'_SINGLETONS_GROUP_': ['COUNTER',
   'run',
   'luminosityBlock',
   'event'],
  'Electron': ['nElectron',
   'charge',
   'dxy',
   'dxyErr',
   'dz',
   'dzErr',
   'eta',
   'mass',
   'pfRelIso03_all',
   'phi',
   'pt'],
  'MET': ['nMET', 'phi', 'pt'],
  'Muon': ['nMuon',
   'charge',
   'dxy',
   'dxyErr',
   'dz',
   'dzErr',
   'eta',
   'mass',
   'pfRelIso03_all',
   'pfRelIso04_all',
   'phi',
   'pt'],
  'PV': ['nPV', 'npvs', 'x', 'y', 'z']},
 '_MAP_DATASETS_TO_COUNTERS_': {'_SINGLETONS_GROUP_': '_SINGLETONS_GROUP_/COUNTER',
  'run': '_SINGLETONS_GROUP_/COUNTER',
  'luminosityBlock': '_SINGLETONS_GROUP_/COUNTER',
  'event': '_SINGLETONS_GROUP_/COUNTER',
  'Electron': 'Electron/nElectron',
  'Electron/charge': 'Electron/nElectron',
  'Electron/dxy': 'Electron/nElectron',
  'Electron/dxyErr': 'Electron/nElectron',
  'Electron/dz': 'Electron/nElectron',
  'Electron/dzErr': 'Electron/nElectron',
  'Electron/eta': 'Electron/nElectron',
  'Electron/mass': 'Electron/

# Scratch code

Just a bunch of test code when I was trying to figure this all out. 

In [27]:
d = {}
groups_to_datasets = {}

counters = []

for field in events.fields:
    
    print(field)
    
    d[field] = []
    groups_to_datasets[field] = []
    
    counters.append(f'n{field}')
    
    for v in events[field].fields:
        groups_to_datasets[field].append(v)
        
        key = f"{field}/{v}"
        
        x = events[field][v]
        
        #print(v)
        
        print(f"\t{v}   {x.ndim}")
        
        if x.ndim==1:
            dtype = x.layout.format
            x = ak.to_numpy(x)

        else:
            dtype = x.layout.content.format
            x = ak.flatten(x).to_numpy()


        d[key] = x

AttributeError: 'Model_TTree_v20' object has no attribute 'fields'

In [ ]:
!ls -ltr

In [ ]:
ak.num(events['Muon']['pt'])

In [ ]:
events.luminosityBlock

In [ ]:
d

In [ ]:
x = events['MET']['pt']

print(x.ndim)
print(events['Muon']['pt'].ndim)
print(events['MET']['pt'].ndim)

In [ ]:
#x.layout
layout = events['Electron']['pt'].layout

In [ ]:
layout.content.format

In [ ]:
x = events['MET']['pt']

x.layout.format

In [ ]:
x = events['MET']['pt']

x.layout

In [ ]:
x = events['MET']['pt']

try:
    x = ak.flatten(x)
except:
    1
x = ak.to_numpy(x)

print(type(x))

# Testing out the loop way

In [ ]:
data = hepfile.initialize()
data

In [ ]:
hepfile.create_group(data,group_name='muon',counter='nmuon')
hepfile.create_dataset(data,group='muon',dset_name=['px','py','pz'],dtype=float)

hepfile.create_dataset(data,dset_name=['luminosity_block'],dtype=int)

bucket = hepfile.create_single_bucket(data)

In [ ]:
data

In [ ]:
nevents = 100

for i in range(nevents):
    nmuon = np.random.randint(0,5)
    bucket['muon/nmuon'] = nmuon
    bucket['muon/px'] = np.random.random(nmuon).tolist()
    bucket['muon/py'] = np.random.random(nmuon).tolist()
    bucket['muon/pz'] = np.random.random(nmuon).tolist()
    
    bucket['luminosity_block'] = np.random.randint(100,10000)
    
    hepfile.pack(data,bucket)

hepfile.write_to_file('awkward_write_test_LOOP_FILL.h5', data, verbose=True)
 

In [ ]:
data,bucket = hepfile.load('awkward_write_test_LOOP_FILL.h5')

In [ ]:
data['luminosity_block']